<a href="https://colab.research.google.com/github/nyongja/basic-of-reinforcement-learning/blob/main/Contextual_Policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 52kB/s 
     |████████████████████████████████| 3.2MB 53.3MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 491kB 50.6MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## 콘텍스트 밴딧
각각 손잡이가 4개인 3개의 밴딧을 사용.  
각각의 밴딧은 각각의 손잡이에 대해 다른 성공 확률을 가지고 있으므로 최고의 결과를 얻기 위해서는 다른 액션이 요구.  
2장의 코드와 유사하게, pullBanadit 함수는 표준정규분포로부터 랜덤한 수를 발생시킴.  
구자 낮을수록 양의 보상 반환될 가능성이 큼.  
에이전트가 주어진 밴딧에 대해 가장 자주 양의 보상을 주는 손잡이를 항상 선택하는 방법을 학습하는 것이 목표.

In [9]:
class contextual_bandit() :
  def __init__(self) :
    self.state = 0
    # 밴딧들의 손잡이 목록을 작성. 각 밴딧은 각각 손잡이 4, 2, 1이 최적.
    self.bandits = np.array([[0.2, 0, -0.0, -5], [0.1, -5, 1, 0.25], [-5, 5, 5, 5]])
    self.num_bandits = self.bandits.shape[0]
    self.num_actions = self.bandits.shape[1]


  def getBandit(self) :
    # 각각의 에피소드에 대해 랜덤한 상태를 반환
    self.state = np.random.randint(0, len(self.bandits))
    return self.state

  def pullArm(self, action) :
    # 랜덤한 수를 얻는다.
    bandit = self.bandits[self.state, action]
    result = np.random.randn(1)
    if result > bandit :
      # 양의 보상을 반환
      return 1
    else :
      # 음의 보상을 반환
      return -1

## 정책 기반 에이전트
입력으로 현재의 상태를 받아 액션을 반환하는 게 전부.  
에이전트는 환경의 상태를 조건으로 삼아 액션을 취하게 됨.  
에이전트는 1개의 가중치 세트를 이용하는데, 각 가중치의 값은 주어진 밴딧의 특정 손잡이를 선택할 때 반환되는 값의 추정값.  
policy gradient를 통해 선택된 액션에 대해 더 큰 보상을 받는 쪽으로 이동하도록 에이전트를 업데이트함.   

In [3]:
class agent() :
  def __init__(self, lr, s_size, a_size) :
    # 네트워크의 피드포워드 부분. 에이전트는 상태를 받아서 액션을 출력
    self.state_in = tf.placeholder(shape=[1], dtype = tf.int32)
    state_in_OH = slim.one_hot_encoding(self.state_in, s_size)
    output = slim.fully_connected(state_in_OH, a_size,
                                  biases_initializer=None, activation_fn=tf.nn.sigmoid,
                                  weights_initializer=tf.ones_initializer())
    self.output = tf.reshape(output, [-1])
    self.chosen_action = tf.argmax(self.output, 0)

    # 학습 과정을 구현함
    # 비용을 계산하기 위해 보상과 선택된 액션을 네트워크에 피드하고,
    # 네트워크를 업데이트하는 데에 이를 이용함.
    self.reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
    self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
    self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
    self.loss = -(tf.log(self.responsible_weight) * self.reward_holder)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
    self.update = optimizer.minimize(self.loss)

## 에이전트 학습시키기
에이전트는 환경의 상태를 알아내고, 액션을 취하고, 보상을 받음으로써 학습할 것.  
이 세가지를 이용해, 주어진 상태에서 시간의 흐름에 따라 최고의 보상을 받을 수 있는 액션을 더 자주 선택할 수 있도록 네트워크를 적절하게 업데이트하는 방법을 알 수 있음

In [12]:
# 텐서플로 그래프를 리셋한다.
tf.reset_default_graph()

# 밴딧을 로드한다.
cBandit = contextual_bandit()
# 에이전트를 로드한다.
myAgent = agent(lr = 0.001, s_size = cBandit.num_bandits, a_size = cBandit.num_actions)
# 네트워크 내부를 들여다보기 위해 평가할 가중치
weights = tf.trainable_variables()[0]

# 에이전트를 학습시킬 전체 에피소드 수 설정
total_episodes = 10000
# 밴딧에 대한 점수판을 0으로 설정
total_reward = np.zeros([cBandit.num_bandits, cBandit.num_actions])
# 랜덤한 액션을 취할 가능성 설정
e = 0.1

init = tf.global_variables_initializer()

# 텐서플로 그래프 론칭
with tf.Session() as sess :
  sess.run(init)
  i = 0
  while i < total_episodes :
    # 환경으로부터 상태 가져오기
    s = cBandit.getBandit()
    # 네트워크로부터 랜덤한 액션 또는 하나의 액션을 선택
    if np.random.rand(1) < e :
      action = np.random.randint(cBandit.num_actions)
    else :
      action = sess.run(myAgent.chosen_action, feed_dict = {myAgent.state_in:[s]})
    
    # 주어진 밴딧에 대해 액션을 취한 데 대한 보상을 얻는다.
    reward = cBandit.pullArm(action)

    # 네트워크를 업데이트한다.
    feed_dict = {myAgent.reward_holder : [reward],
                 myAgent.action_holder : [action],
                 myAgent.state_in : [s]}
    _, ww = sess.run([myAgent.update, weights], feed_dict = feed_dict)

    # 보상의 총계 업데이트
    total_reward[s, action] += reward
    if i % 500 == 0 :
      print("Mean reward for each of the " + str(cBandit.num_bandits) + 
            " bandits : " + str(np.mean(total_reward, axis = 1)))
    i += 1

for a in range(cBandit.num_bandits) :
  print("The agent thinks action " + str(np.argmax(ww[a]) + 1) + " for bandit " + str(a+1) + " is the most promising....")
  if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]) :
    print("...and it was right!")
  else :
    print("...and it was wrong!")

Mean reward for each of the 3 bandits : [0.   0.25 0.  ]
Mean reward for each of the 3 bandits : [31.75 38.5  33.5 ]
Mean reward for each of the 3 bandits : [71.75 79.   65.5 ]
Mean reward for each of the 3 bandits : [106.5  119.25 103.  ]
Mean reward for each of the 3 bandits : [146.   159.   137.25]
Mean reward for each of the 3 bandits : [184.75 194.75 174.25]
Mean reward for each of the 3 bandits : [225.75 233.5  206.5 ]
Mean reward for each of the 3 bandits : [266.75 272.25 238.25]
Mean reward for each of the 3 bandits : [304.75 313.5  273.  ]
Mean reward for each of the 3 bandits : [343.75 353.25 306.75]
Mean reward for each of the 3 bandits : [384.75 392.   339.  ]
Mean reward for each of the 3 bandits : [421.75 428.25 376.25]
Mean reward for each of the 3 bandits : [463.5  466.   407.75]
Mean reward for each of the 3 bandits : [499.75 502.25 448.25]
Mean reward for each of the 3 bandits : [533.75 537.25 488.75]
Mean reward for each of the 3 bandits : [570.75 573.   521.5 ]
Mean